In [1]:
import os
from pathlib import Path
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [2]:
if iskaggle:
    path = Path('../input/us-patent-phrase-to-phrase-matching')
    ! pip install -q datasets

In [3]:
!ls {path}

sample_submission.csv  test.csv  train.csv


In [4]:
import pandas as pd

In [5]:
df = pd.read_csv(path/'train.csv')

In [6]:
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [7]:
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,8d135da0b55b8c88,component composite coating,composition,H01
freq,1,152,24,2186


In [8]:
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor

In [9]:
df.input.head()

0    TEXT1: A47; TEXT2: abatement of pollution; ANC...
1    TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2    TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3    TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4    TEXT1: A47; TEXT2: forest region; ANC1: abatement
Name: input, dtype: object

In [10]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)

In [11]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [12]:
model_nm = 'microsoft/deberta-v3-small'

In [13]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [14]:
tokz.tokenize("A platypus is an ornithorhynchus anatinus")

['▁A',
 '▁platypus',
 '▁is',
 '▁an',
 '▁or',
 'ni',
 'tho',
 'rhynch',
 'us',
 '▁an',
 'at',
 'inus']

In [15]:
def tok_func(x): return tokz(x["input"])

In [16]:
tok_ds = ds.map(tok_func, batched=True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [17]:
row = tok_ds[0]
row['input'], row['input_ids']

('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2])

In [18]:
tokz.vocab['▁of']

265

In [19]:
tok_ds = tok_ds.rename_columns({'score':'labels'})

In [20]:
eval_df = pd.read_csv(path/'test.csv')
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,hybrid bearing,inorganic photoconductor drum,G02
freq,1,2,1,3


In [21]:
import numpy as np, matplotlib.pyplot as plt

In [22]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [23]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [24]:
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [25]:
from transformers import TrainingArguments, Trainer

In [26]:
bs = 128
epochs = 4

In [27]:
lr = 8e-5

In [28]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [29]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-29-48e257fff1ad>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],


In [30]:
trainer.train();

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.030487,0.797361
2,No log,0.022620,0.819524
3,0.039700,0.022505,0.828560
4,0.039700,0.022785,0.830174


In [31]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[ 0.6015625 ],
       [ 0.72802734],
       [ 0.54638672],
       [ 0.29516602],
       [-0.00723648],
       [ 0.50537109],
       [ 0.5       ],
       [-0.00836945],
       [ 0.25830078],
       [ 1.10058594],
       [ 0.2512207 ],
       [ 0.30053711],
       [ 0.73681641],
       [ 0.89160156],
       [ 0.73388672],
       [ 0.36621094],
       [ 0.31494141],
       [-0.03521729],
       [ 0.64990234],
       [ 0.38769531],
       [ 0.41113281],
       [ 0.2512207 ],
       [ 0.12536621],
       [ 0.24157715],
       [ 0.54589844],
       [-0.04272461],
       [-0.04278564],
       [-0.0168457 ],
       [-0.04940796],
       [ 0.61425781],
       [ 0.37597656],
       [-0.00242233],
       [ 0.70654297],
       [ 0.51904297],
       [ 0.39697266],
       [ 0.2421875 ]])

In [32]:
preds = np.clip(preds, 0,1)

In [33]:
preds

array([[0.6015625 ],
       [0.72802734],
       [0.54638672],
       [0.29516602],
       [0.        ],
       [0.50537109],
       [0.5       ],
       [0.        ],
       [0.25830078],
       [1.        ],
       [0.2512207 ],
       [0.30053711],
       [0.73681641],
       [0.89160156],
       [0.73388672],
       [0.36621094],
       [0.31494141],
       [0.        ],
       [0.64990234],
       [0.38769531],
       [0.41113281],
       [0.2512207 ],
       [0.12536621],
       [0.24157715],
       [0.54589844],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.61425781],
       [0.37597656],
       [0.        ],
       [0.70654297],
       [0.51904297],
       [0.39697266],
       [0.2421875 ]])

In [34]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds
})

submission.to_csv('submission.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1006